# CADS API Python client Tests

In [1]:
import os
import xarray as xr

import cads_api_client

In [2]:
api_url = os.getenv("CADS_API_ROOT_URL", "http://cds2-dev.bopen.eu/api")
api_url = os.getenv("CADS_API_ROOT_URL", "http://localhost:8080/api")

api_url

'http://localhost:8080/api'

## Client instantiation

The client exposes the APIs for:
- Catalogue exploration (see **Section 1**) 
- Data retrieval (see **Section 2**)

In [3]:
client = cads_api_client.ApiClient(api_url, token="mysecretpat")
client

ApiClient()

## 1. Catalogue Exploration


### 1.1 Collections

**Objective**: verify the access to the list of **collections** and their description.
<hr>

_**Expected result**: correct instatiation of collections object._
_If `collections.response` returns 200 status code, the request is succesfull and the test can proceed._

In [4]:
collections = client.collections()
collections.response

<Response [200]>

_**Expected result**: list of all available collections._

In [5]:
collections.collection_ids()

['cams-global-reanalysis-eac4-monthly',
 'reanalysis-era5-single-levels',
 'reanalysis-era5-land',
 'reanalysis-era5-pressure-levels',
 'reanalysis-era5-land-monthly-means',
 'derived-near-surface-meteorological-variables']

### 1.2 Collection

**Objective**: verify the access to the list of **collection** and its description
<hr>

_**Expected result**: correct instatiation of the collection object._
_If `collection.response` returns 200 status code, the request is succesfull and the test can proceed._

In [6]:
collection = client.collection("reanalysis-era5-pressure-levels")
collection

Collection(response=<Response [200]>, headers={'PRIVATE-TOKEN': 'mysecretpat'})

_**Expected result**: JSON of the collection response describing the collection, containing the keys:_
- _`id`_
- _`title`_
- _`description`_

In [7]:
collection.json

{'type': 'Collection',
 'id': 'reanalysis-era5-pressure-levels',
 'stac_version': '1.0.0',
 'title': 'ERA5 hourly data on pressure levels from 1959 to present',
 'description': '**ERA5** is the fifth generation ECMWF reanalysis for the global climate and weather for the past 4 to 7 decades.\nCurrently data is available from 1950, with Climate Data Store entries for 1950-1978 (preliminary back extension) and from 1959 onwards (final release plus timely updates, this page).\nERA5 replaces the ERA-Interim reanalysis.\n\nReanalysis combines model data with observations from across the world into a globally complete and consistent dataset using the laws of physics. This principle, called data assimilation, is based on the method used by numerical weather prediction centres, where every so many hours (12 hours at ECMWF) a previous forecast is combined with newly available observations in an optimal way to produce a new best estimate of the state of the atmosphere, called analysis, from which

## 2. Data Retrieval

The retrieval can be done using the high-level function `client.retrieve` (see **Section 2.1**). It performs the submission, the monitoring and the download.

Alternatively it can be done using the low-level API (see **Section 3** Data Retrieval: advanced usage): 
- **Section 3.1**: `client.submit` submits the request
- **Section 3.2**: `remote.status` monitors the request
- **Section 3.3**: `remote.download` downloads the result


### 2.1  Retrieval: small data request

**Objective**: verify the capabilities of the client to retrieve data.
<hr>


**`client.retrieve`** function is blocking: 
- submits the request
- waits until the request is complete
- downloads the data


_**Expected result**: the client submits the request; when the process is complete, the client downloads the file._

In [8]:
%pdb
output_path = client.retrieve(
    collection_id="reanalysis-era5-pressure-levels",
    product_type="reanalysis", 
    variable="temperature", 
    pressure_level="1", 
    year="1971", 
    month="01", 
    day="25", 
    time="06:00",
    target="test01.grib",
)

output_path

Automatic pdb calling has been turned ON


21082651853192445323832284060735019160.grib: 0.00B [00:00, ?B/s]

URL http://localhost:8080/api/storage/v1/cache/21082651853192445323832284060735019160.grib: <?xml version="1.0" encoding="UTF-8"?>
<Error><Code>AccessDenied</Code><Message>Access Denied.</Message><Key>21082651853192445323832284060735019160.grib</Key><BucketName>cache</BucketName><Resource>/cache/21082651853192445323832284060735019160.grib</Resource><RequestId>1729EDC47E69A4BC</RequestId><HostId>0a138fd5-5520-4593-a272-0594f0c8dd17</HostId></Error>


HTTPError: 403 Client Error: Forbidden for url: http://localhost:8080/api/storage/v1/cache/21082651853192445323832284060735019160.grib

> /usr/local/Caskroom/miniconda/base/envs/DEVELOP/lib/python3.10/site-packages/requests/models.py(1021)raise_for_status()
   1019 
   1020         if http_error_msg:
-> 1021             raise HTTPError(http_error_msg, response=self)
   1022 
   1023     def close(self):

ipdb> c


In [ ]:
ls -l $output_path

_**Expected output**: data consistent with the request_
- _one variable: `t` (temperature)_
- _two dimensions: `latitude` and `longitude`_
- _`time` coordinate has length 1_

In [ ]:
ds = xr.open_dataset(output_path)
ds

## 3 Data Retrieval: advanced API


### 3.1 Request Submission: big data request

**Objective:** verify the capability of the client to submit a request.
<hr>



**`client.submit`** is a non-blocking function. It returns a remote object that allows to monitor the process.


_**Expected result**: the client submits the request and returns a remote object that allows to monitor the process._

In [9]:
collection = client.collection("reanalysis-era5-pressure-levels")
remote = collection.submit( 
    product_type="reanalysis", 
    variable="temperature", 
    pressure_level="1", 
    year="1971", 
    month=['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12'],
    day=[
        '01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12',
        '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24',
        '25', '26', '27', '28', '29', '30', '31',
    ],
    time="06:00",
    target="test02.grib",
)
remote.request_uid

'd945ec06-6e07-4541-acfd-b2395ff96762'

### 3.2 Request Monitoring

**Objectives**: verify the capability of the client to monitor the request
<hr>

**`remote.status`** allows to monitor the process status

_**Expected result**: returns the updated status of the request: failed, successful, running_

In [10]:
remote.status

'running'

**`client.get_requests`** returns the list of requests submitted 

_**Expected result**: IDs of the submitted requests._

In [11]:
requests = client.get_requests()
requests.job_ids()

['d945ec06-6e07-4541-acfd-b2395ff96762',
 '3f56b5c3-ccae-4cfe-bc90-2717f7eb5300',
 '6cc24961-d2e6-4a43-85ed-e7d6903bf6ba',
 '2f89a089-c4fb-4ce8-bfcf-a857530f0fb7',
 'b2ad9a77-5f3a-4c70-ae32-f9c8d16a3786',
 'af04cf40-4972-4393-ac91-d26e3d1fc9eb',
 '35371fe2-981b-4819-880e-e69b8f1202d3',
 '0f967266-9a61-43a8-a5d7-36aa2f6f0192']

_**Expected result**: `True`, that is the remote ID, `remote.request_uid`, is in the list of the submitted requests._

In [12]:
remote.request_uid in requests.job_ids()

True

### 3.3 Data download

**Objectives**: verify the capability of the client to download the data

**`remote.download`** is blocking: 
- waits until the requests is completed
- downloads the data requested
- returns the output path

_**Expected result**: the file is downloaded and saved in `$output_path`._

In [ ]:
output_path = remote.download("test02.grib")
output_path

In [ ]:
ls -l $ouput_path

_**Expected output**: data consistent with the request:_
- _one variable: `t` (temperature),_
- _three dimensions: `time` (365), `latitude` (721) and `longitude` (1440)_

In [ ]:
ds = xr.open_dataset(output_path)
ds

## 4 Error Handling

**Objectives**: verify how errors are handled through error messages and codes.
<hr>

### 4.1 Wrong URL

_**Expected output**: 404 Client Error: Not Found for url._

In [13]:
client = cads_api_client.ApiClient(f"{api_url}_1")
client.collections()

HTTPError: 502 Server Error: Bad Gateway for url: http://localhost:8080/api_1/catalogue/v1/collections

> /usr/local/Caskroom/miniconda/base/envs/DEVELOP/lib/python3.10/site-packages/requests/models.py(1021)raise_for_status()
   1019 
   1020         if http_error_msg:
-> 1021             raise HTTPError(http_error_msg, response=self)
   1022 
   1023     def close(self):

ipdb> c


### 4.2 Missing collection

_**Expected output**: 404 Client Error: Not Found for url._

In [ ]:
client = cads_api_client.ApiClient(api_url)
client.collection("missing_collection")

### 4.3 Unknown job

_**Expected output**: 404 Client Error: Not Found for url._

In [ ]:
status_info = client.get_request("ffffffff-4455-6677-8899-aabbccddeeff")

### 4.4 Wrong request

_**Expected output**: `collection.submit` does not raise errors._


In [ ]:
client = cads_api_client.ApiClient(api_url)
collection = client.collection("reanalysis-era5-pressure-levels")
remote = collection.submit( 
    target="output.grib",
    product_type="reanalysis", 
    variable="temperature", 
    pressure_level="1", 
    year="2222", 
    month="01", 
    day="25", 
    time="06:00",
    format="grib", 
)

_**Expected output**: `remote.wait_on_result` raises a ProcessingFailedError-_

In [ ]:
remote.wait_on_result()

_**Expected output**: result status code is 400._

In [ ]:
results = remote.make_results()

In [ ]:
results.status_code

_**Expected output**:_
- _`result.json` describes the error in the fields: "type", "title" and "detail"_
- _"detail" contains the error traceback_

In [ ]:
results.json